**bold text**Distance Estimation between two vehicles. **bold text**


In [ ]:
import cv2
import torch
import numpy as np
from collections import deque
from pytube import YouTube

# Load YOLOv5 model from PyTorch Hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Initialize tracker
class SimpleTracker:
    def __init__(self):
        self.trackers = []

    def update(self, detections):
        for tracker in self.trackers:
            tracker['lost'] += 1

        for detection in detections:
            x1, y1, x2, y2, conf, cls = detection
            matched = False
            for tracker in self.trackers:
                if self.iou(tracker['bbox'], [x1, y1, x2, y2]) > 0.5:
                    tracker['bbox'] = [x1, y1, x2, y2]
                    tracker['lost'] = 0
                    matched = True
                    break
            if not matched:
                self.trackers.append({'bbox': [x1, y1, x2, y2], 'lost': 0})

        self.trackers = [t for t in self.trackers if t['lost'] < 5]

        return self.trackers

    def iou(self, bbox1, bbox2):
        x1, y1, x2, y2 = bbox1
        x1_, y1_, x2_, y2_ = bbox2

        xi1, yi1 = max(x1, x1_), max(y1, y1_)
        xi2, yi2 = min(x2, x2_), min(y2, y2_)

        inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

        bbox1_area = (x2 - x1) * (y2 - y1)
        bbox2_area = (x2_ - x1_) * (y2_ - y1_)

        union_area = bbox1_area + bbox2_area - inter_area

        return inter_area / union_area

# Calculate Euclidean distance between two vehicles
def calculate_distance(vehicle1, vehicle2):
    pos1 = np.array(vehicle1['center'])
    pos2 = np.array(vehicle2['center'])
    distance = np.linalg.norm(pos1 - pos2)
    return distance

# Download video from YouTube
def download_youtube_video(youtube_url, output_path):
    yt = YouTube(youtube_url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
    stream.download(output_path=output_path, filename='video.mp4')
    return f"{output_path}/video.mp4"

# Process video frames
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    tracker = SimpleTracker()
    vehicle_tracks = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        detections = []
        for *box, conf, cls in results.xyxy[0].cpu().numpy():
            if cls == 2:  # Only consider cars
                detections.append(box + [conf, cls])

        tracks = tracker.update(detections)

        for i, track in enumerate(tracks):
            x1, y1, x2, y2 = track['bbox']
            center = [(x1 + x2) / 2, (y1 + y2) / 2]
            track['center'] = center
            if i not in vehicle_tracks:
                vehicle_tracks[i] = deque(maxlen=5)
            vehicle_tracks[i].append(center)
            if len(vehicle_tracks[i]) > 1:
                track['velocity'] = np.mean(np.diff(vehicle_tracks[i], axis=0), axis=0)
            else:
                track['velocity'] = np.array([0, 0])

        for i, track in enumerate(tracks):
            for j, other_track in enumerate(tracks):
                if i != j:
                    distance = calculate_distance(track, other_track)
                    print(f"Distance between vehicle {i} and vehicle {j}: {distance:.2f} pixels")

    cap.release()

if __name__ == "__main__":
    youtube_url = 'https://www.youtube.com/watch?v=MNn9qKG2UFI'
    video_path = download_youtube_video(youtube_url, './')
    process_video(video_path)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 201.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 172.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 9.7s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-7-3 Python-3.10.12 torch-2.3.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 137MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Streaming output truncated to the last 5000 lines.
Distance between vehicle 9 and vehicle 8: 97.48 pixels
Distance between vehicle 9 and vehicle 10: 38.55 pixels
Distance between vehicle 10 and vehicle 0: 310.17 pixels
Distance between vehicle 10 and vehicle 1: 344.67 pixels
Distance between vehicle 10 and vehicle 2: 80.28 pixels
Distance between vehicle 10 and vehicle 3: 180.82 pixels
Distance between vehicle 10 and vehicle 4: 250.52 pixels
Distance between vehicle 10 and vehicle 5: 115.16 pixels
Distance between vehicle 10 and vehicle 6: 33.69 pixels
Distance between vehicle 10 and vehicle 7: 62.50 pixels
Distance between vehicle 10 and vehicle 8: 94.87 pixels
Distance between vehicle 10 and vehicle 9: 38.55 pixels
Distance between vehicle 0 and vehicle 1: 296.98 pixels
Distance between vehicle 0 and vehicle 2: 276.83 pixels
Distance between vehicle 0 and vehicle 3: 228.23 pixels
Distance between vehicle 0 and vehicle 4: 160.38 pixels
Distance between vehicle 0 and vehicle 5: 196.58 

KeyboardInterrupt: 

In [ ]:
pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00


Providing warning below the defined threshold.**bold text**

In [ ]:
import cv2
import torch
import numpy as np
from collections import deque
from pytube import YouTube

# Load YOLOv5 model from PyTorch Hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Initialize tracker
class SimpleTracker:
    def __init__(self):
        self.trackers = []

    def update(self, detections):
        for tracker in self.trackers:
            tracker['lost'] += 1

        for detection in detections:
            x1, y1, x2, y2, conf, cls = detection
            matched = False
            for tracker in self.trackers:
                if self.iou(tracker['bbox'], [x1, y1, x2, y2]) > 0.5:
                    tracker['bbox'] = [x1, y1, x2, y2]
                    tracker['lost'] = 0
                    matched = True
                    break
            if not matched:
                self.trackers.append({'bbox': [x1, y1, x2, y2], 'lost': 0})

        self.trackers = [t for t in self.trackers if t['lost'] < 5]

        return self.trackers

    def iou(self, bbox1, bbox2):
        x1, y1, x2, y2 = bbox1
        x1_, y1_, x2_, y2_ = bbox2

        xi1, yi1 = max(x1, x1_), max(y1, y1_)
        xi2, yi2 = min(x2, x2_), min(y2, y2_)

        inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

        bbox1_area = (x2 - x1) * (y2 - y1)
        bbox2_area = (x2_ - x1_) * (y2_ - y1_)

        union_area = bbox1_area + bbox2_area - inter_area

        return inter_area / union_area

# Calculate Euclidean distance between two vehicles
def calculate_distance(vehicle1, vehicle2):
    pos1 = np.array(vehicle1['center'])
    pos2 = np.array(vehicle2['center'])
    distance = np.linalg.norm(pos1 - pos2)
    return distance

# Calculate Time to Collision (TTC)
def calculate_ttc(vehicle1, vehicle2):
    pos1 = np.array(vehicle1['center'])
    pos2 = np.array(vehicle2['center'])
    vel1 = np.array(vehicle1['velocity'])
    vel2 = np.array(vehicle2['velocity'])

    relative_pos = pos2 - pos1
    relative_vel = vel2 - vel1

    relative_speed = np.dot(relative_pos, relative_vel) / np.linalg.norm(relative_pos)

    if relative_speed <= 0:
        return float('inf')

    relative_distance = np.linalg.norm(relative_pos)
    ttc = relative_distance / relative_speed
    return ttc

# Check if warning is needed
def is_warning_needed(time_to_collision):
    warning_threshold = 0.7  # Time in seconds

    if time_to_collision < warning_threshold:
        print("WARNING: Potential collision imminent!")
        return True
    else:
        return False

# Download video from YouTube
def download_youtube_video(youtube_url, output_path):
    yt = YouTube(youtube_url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
    stream.download(output_path=output_path, filename='video.mp4')
    return f"{output_path}/video.mp4"

# Process video frames
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    tracker = SimpleTracker()
    vehicle_tracks = {}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)
        detections = []
        for *box, conf, cls in results.xyxy[0].cpu().numpy():
            # Consider all types of vehicles: car, motorcycle, bus, truck
            if cls in [2, 3, 5, 7]:
                detections.append(box + [conf, cls])

        tracks = tracker.update(detections)

        for i, track in enumerate(tracks):
            x1, y1, x2, y2 = track['bbox']
            center = [(x1 + x2) / 2, (y1 + y2) / 2]
            track['center'] = center
            if i not in vehicle_tracks:
                vehicle_tracks[i] = deque(maxlen=5)
            vehicle_tracks[i].append(center)
            if len(vehicle_tracks[i]) > 1:
                track['velocity'] = np.mean(np.diff(vehicle_tracks[i], axis=0), axis=0)
            else:
                track['velocity'] = np.array([0, 0])

        for i, track in enumerate(tracks):
            for j, other_track in enumerate(tracks):
                if i != j:
                    ttc = calculate_ttc(track, other_track)
                    if is_warning_needed(ttc):
                        distance = calculate_distance(track, other_track)
                        print(f"Warning: TTC between vehicle {i} and vehicle {j} is below threshold: {ttc:.2f} seconds")
                        print(f"Distance between vehicle {i} and vehicle {j}: {distance:.2f} pixels")

    cap.release()

if __name__ == "__main__":
    youtube_url = 'https://www.youtube.com/watch?v=MNn9qKG2UFI'
    video_path = download_youtube_video(youtube_url, './')
    process_video(video_path)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-7-3 Python-3.10.12 torch-2.3.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Streaming output truncated to the last 5000 lines.
Distance between vehicle 12 and vehicle 6: 56.56 pixels
Distance between vehicle 12 and vehicle 8: 33.15 pixels
Distance between vehicle 12 and vehicle 9: 18.65 pixels
Distance between vehicle 13 and vehicle 15: 11.19 pixels
Distance between vehicle 15 and vehicle 13: 11.19 pixels
Distance between vehicle 8 and vehicle 11: 19.57 pixels
Distance between vehicle 11 and vehicle 8: 19.57 pixels
Distance between vehicle 8 and vehicle 11: 20.91 pixels
Distance between vehicle 11 and vehicle 8: 20.91 pixels
Distance between vehicle 8 and vehicle 11: 21.43 pixels
Distance between vehicle 11 and vehicle 8: 21.43 pixels
Distance between vehicle 8 and vehicle 11: 22.23 pixels
Distance between vehicle 11 and vehicle 8: 22.23 pixels
Distance between vehicle 12 and vehicle 15: 1.59 pixels
Distance between vehicle 15 and vehicle 12: 1.59 pixels
Distance between vehicle 12 and vehicle 18: 21.21 pixels
Distance between vehicle 15 and vehicle 18: 19.66 